<a href="https://colab.research.google.com/github/Ldoun/tensorflow_study/blob/master/kaggle_real_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# API Token 다운받기
# 다운받은 API Token 업로드 하기
!pip install kaggle
from google.colab import files
files.upload()
# json 파일 옮겨주기

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# json 파일 제대로 업로드 됐는지 확인
!ls -1ha kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list
# 데이터셋 다운로드 받기 - 링크는 그 대회 'Data'에 있음
! kaggle competitions download -c nlp-getting-started
# 다운로드 된 것들 다 보기 
!ls

Saving kaggle.json to kaggle.json
kaggle.json
ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         15           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        265           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2839           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      21611            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       5108            True  
connectx                                       2030-01-01 00:00:00  Getting Started  K

In [ ]:
! kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 65.0MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 140MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#전처리용
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train=pd.read_csv('train.csv')

In [ ]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
# train['keyword].count(None) 다른 값 나오
print('전체 keyword 수',len(train['keyword']))
print('keyword 수:',(len(train['keyword'])-train['keyword'].isnull().sum()))

전체 keyword 수 7613
keyword 수: 7552


In [ ]:
#loction 없는게 너무 많음
print('전체 location 수',len(train['location']))
print('location 수:',(len(train['location'])-train['location'].isnull().sum()))

전체 location 수 7613
location 수: 5080


In [ ]:
print('전체 text 수',len(train['text']))
print('text 수:',(len(train['text'])-train['text'].isnull().sum()))

전체 text 수 7613
text 수: 7613


In [ ]:
print("max:", max(len(i)for i in train['text']))
print("min:", min(len(i)for i in train['text']))
print("mid:", sum(len(i) for i in train['text'])/len(train['text']))

max: 157
min: 7
mid: 101.03743596479706


In [ ]:
tokenizer=Tokenizer(10000)
tokenizer.fit_on_texts(train['text'])
seq=tokenizer.texts_to_sequences(train['text'])
print("문자 개수",len(tokenizer.word_index))

문자 개수 22700


In [ ]:
for i,word in enumerate(train['keyword']):
  if isinstance(word,str)==False:
    train['keyword'][[i]]=''
    print(word)  

In [ ]:
tokenizer_keyword=Tokenizer(500)
tokenizer.fit_on_texts(train['keyword'])
seq_keyword=tokenizer.texts_to_sequences(train['keyword'])
print(seq_keyword)
print("문자 개수",max(seq_keyword))

[[34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [34], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [162], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [217], [163, 245], [163, 245], [163, 245], [163, 245], [163, 245], [163, 245], [163, 245], [163, 245], [163, 245], [163,

In [ ]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Embedding,Flatten,Dense,Conv1D,LeakyReLU,MaxPool1D,concatenate,Dropout
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
seq=pad_sequences(seq,101)

In [ ]:
seq_keyword=pad_sequences(seq,2)

In [ ]:
#Text daa
input_seq=Input((101,),name='text')
embedded_seq=Embedding(10000,64)(input_seq)
dropout_emb = Dropout(0.5)(embedded_seq)
conv1=Conv1D(64,3,padding='same')(dropout_emb)
Activation=LeakyReLU()(conv1)
pooling1=MaxPool1D()(Activation)

conv2=Conv1D(64,4,padding='same')(dropout_emb)
Activation=LeakyReLU()(conv2)
pooling2=MaxPool1D()(Activation)

conv3=Conv1D(64,5,padding='same')(dropout_emb)
Activation=LeakyReLU()(conv3)
pooling3=MaxPool1D()(Activation)

con=concatenate([pooling1,pooling2,pooling3])
flat=Flatten()(con)
hidden=Dense(64)(flat)



#keyword data
input_keyword=Input((2,),name='keyword')
embedded_seq=Embedding(len(tokenizer.word_index)+1,4)(input_keyword)
dropout_emb = Dropout(0.5)(embedded_seq)
flat=Flatten()(dropout_emb)
hidden_1=Dense(64)(flat)


combine=concatenate([hidden,hidden_1])
hidden3=Dense(32)(combine)
activ=LeakyReLU()(hidden3)
output=Dense(1,activation=sigmoid)(activ)





In [ ]:
model=Model([input_seq,input_keyword],output)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',)
model.fit({'text':seq,'keyword':seq_keyword},
          train['target'],
          validation_split=0.2,
          epochs=20,
          batch_size=32,
          callbacks=[early_stopping]
          )

Epoch 1/20
191/191 [==============================] - 11s 57ms/step - loss: 0.5885 - acc: 0.6887 - val_loss: 0.4871 - val_acc: 0.7636
Epoch 2/20
191/191 [==============================] - 11s 58ms/step - loss: 0.4153 - acc: 0.8176 - val_loss: 0.4424 - val_acc: 0.8050
Epoch 3/20
191/191 [==============================] - 11s 58ms/step - loss: 0.3537 - acc: 0.8550 - val_loss: 0.4369 - val_acc: 0.8017
Epoch 4/20
191/191 [==============================] - 12s 61ms/step - loss: 0.3093 - acc: 0.8782 - val_loss: 0.4450 - val_acc: 0.8030


In [ ]:
test=pd.read_csv('tdownload.csv')
print(test)

         id keyword location                                               text
0         0     NaN      NaN                 Just happened a terrible car crash
1         2     NaN      NaN  Heard about #earthquake is different cities, s...
2         3     NaN      NaN  there is a forest fire at spot pond, geese are...
3         9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4        11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan
...     ...     ...      ...                                                ...
3258  10861     NaN      NaN  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259  10865     NaN      NaN  Storm in RI worse than last hurricane. My city...
3260  10868     NaN      NaN  Green Line derailment in Chicago http://t.co/U...
3261  10874     NaN      NaN  MEG issues Hazardous Weather Outlook (HWO) htt...
3262  10875     NaN      NaN  #CityofCalgary has activated its Municipal Eme...

[3263 rows x 4 columns]


In [ ]:
for i,word in enumerate(test['keyword']):
  if isinstance(word,str)==False:
    test['keyword'][[i]]=''
    print(word)  

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
test_seq=tokenizer.texts_to_sequences(test['text'])
test_seq_keyword=tokenizer_keyword.texts_to_sequences(test['keyword'])
test_seq=pad_sequences(test_seq,101)
test_seq_keyword=pad_sequences(test_seq_keyword,2)

In [ ]:
pridict=model.predict([test_seq,test_seq_keyword])

In [ ]:
print(len(test))

3263


In [ ]:
sample=pd.read_csv('sample_submission.csv')

In [ ]:
for i,num in enumerate(pridict):
  if num>=0.5:
    pridict[i]=1
  else:
    pridict[i]=0

In [ ]:
i=range(len(pridict))

In [ ]:
import numpy
submission_base = pd.DataFrame()
submission_base['id'] = test['id']
submission_base['target'] = numpy.round(pridict).astype(int)
submission_base.head(10)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
5,12,0
6,21,1
7,22,0
8,27,0
9,29,0


In [ ]:
submission_base.to_csv('sub2.csv')